In [1]:
from langchain_community.vectorstores import FAISS
from configparser import ConfigParser

In [2]:
#load config
config = ConfigParser()
config.read('config.ini')

['config.ini']

In [3]:
import csv
from langchain.schema import Document

documents = []
with open("data.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row_index, row in enumerate(reader):
        index_id = row.get("index_id", f"row_{row_index}")
        for field_name, field_value in row.items():
            if field_name == "index_id":
                continue  # 跳過 index_id 欄位
            if not field_value.strip():
                continue  # 跳過空內容
            doc = Document(
                page_content=field_value.strip(),
                metadata={
                    "index_id": index_id,
                    "field": field_name,
                }
            )
            documents.append(doc)

print(f"產生了 {len(documents)} 筆欄位級 document。")
print(documents[0])
print(documents[1])


產生了 2716 筆欄位級 document。
page_content='泰式打拋醬炒空心菜' metadata={'index_id': '28', 'field': 'name'}
page_content='泰式打拋醬原先的微酸微辣口感，讓人食慾大開。蒜香、辣椒與空心菜快炒後，清脆的空心菜口味層次豐富，是一道非常下飯的泰式料理。' metadata={'index_id': '28', 'field': 'description'}


In [4]:
index_map = {}
with open("data.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        index_id = row["index_id"]
        index_map[index_id] = row

In [5]:
from langchain_openai import AzureOpenAIEmbeddings

embedding_model = AzureOpenAIEmbeddings(
    openai_api_version=config["AzureOpenAI"]["VERSION"],
    azure_deployment=config["AzureOpenAI"]["Embedding_DEPLOYMENT_NAME"],
    azure_endpoint=config["AzureOpenAI"]["ENDPOINT"],
    api_key=config["AzureOpenAI"]["KEY"],
)

In [6]:
# db = FAISS.from_documents(documents, embedding_model)
# 載入 FAISS index
db = FAISS.load_local("faiss_db", embedding_model, "cuisines", allow_dangerous_deserialization=True)

In [7]:
# 儲存 FAISS index
db.save_local("faiss_db", "cuisines")

In [84]:
query = "推薦我一個有米酒、食用油、鮮磨黑胡椒的料理"

In [85]:
result = db.similarity_search_with_score(query, 20)

In [86]:
# 收集所有檢索結果
saved_results = []

for i, (doc, dist) in enumerate(result):
    index_id = doc.metadata["index_id"]
    field = doc.metadata["field"]
    content = doc.page_content
    original_row = index_map.get(index_id, {})

    result_entry = {
        "Result_Rank": i + 1,
        "Score": round(dist, 4),
        "Matched_Field": field,
        "Field_Content": content,
        "Index_ID": index_id,
        "Name": original_row.get("name", ""),
        "Description": original_row.get("description", ""),
        "Ingredients": original_row.get("ingredients", ""),
        "Instructions": original_row.get("instructions", "")
    }

    saved_results.append(result_entry)


In [87]:
contexts = []
for i, result in enumerate(saved_results):
    context = f"Result Rank: {result['Result_Rank']}\n"
    context += f"Score: {result['Score']}\n"
    context += f"Matched Field: {result['Matched_Field']}\n"
    context += f"Field Content: {result['Field_Content']}\n"
    context += f"Index ID: {result['Index_ID']}\n"
    context += f"Name: {result['Name']}\n"
    context += f"Description: {result['Description']}\n"
    context += f"Ingredients: {result['Ingredients']}\n"
    context += f"Instructions: {result['Instructions']}\n\n"

    contexts.append(context)

In [88]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version=config["AzureOpenAI"]["VERSION"],
    azure_deployment=config["AzureOpenAI"]["GPT4o_DEPLOYMENT_NAME"],
    azure_endpoint=config["AzureOpenAI"]["ENDPOINT"],
    api_key=config["AzureOpenAI"]["KEY"],
)

In [89]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

# 封裝成 List[Document]
documents = [Document(page_content=c) for c in contexts]

# 建立 prompt
prompt = ChatPromptTemplate.from_template(
"""
Answer the following question based only on the provided context,
besides the answer, please also provide a list of comparison, 
just list item, not table.

{context}

Question: {input}
請用繁體中文回答
"""
)

# 建立 chain 並傳入
document_chain = create_stuff_documents_chain(llm, prompt)

llm_result = document_chain.invoke(
    {
        "input": query,
        "context": documents
    }
)

In [90]:
for result in saved_results:
    print(f"Rank: {result['Result_Rank']}, Score: {result['Score']}")
    print(f"Matched Field: {result['Matched_Field']}")
    print(f"Field Content: {result['Field_Content']}")
    print(f"Index ID: {result['Index_ID']}")
    print(f"Name: {result['Name']}")
    print(f"Description: {result['Description']}")
    print(f"Ingredients: {result['Ingredients']}")
    print(f"Instructions: {result['Instructions']}")
    print("-" * 40)

Rank: 1, Score: 0.8919000029563904
Matched Field: description
Field Content: 家中必備涮嘴零食，充滿黑胡椒與八角香氣的毛豆，不論是平常當零嘴或下酒菜都非常推薦
Index ID: 306
Name: 蒜香毛豆
Description: 家中必備涮嘴零食，充滿黑胡椒與八角香氣的毛豆，不論是平常當零嘴或下酒菜都非常推薦
Ingredients: ['水 1000毫升', '紅辣椒 1根', '毛豆 400公克', '砂糖 10公克', '玫瑰鹽 10公克', '八角粒 5個', '香蒜粒 10公克', '黑胡椒粉 10公克']
Instructions: ['食材', '毛豆洗淨、辣椒切圈。', '湯鍋裝水，水裡放入八角煮滾，加入1小匙玫瑰鹽和糖煮溶。', '放入毛豆，維持中火煮3-5分鐘。', '取一玻璃盆，毛豆起鍋瀝乾，放入玻璃盆中。', '灑入香蒜粒、紅辣椒、凍頂黑胡椒、1小匙玫瑰鹽與毛豆拌均勻。', '放入冰箱冷藏15分鐘即可食用。']
----------------------------------------
Rank: 2, Score: 0.9271000027656555
Matched Field: ingredients
Field Content: ['蒜頭 5顆', '橄欖油 8毫升', '義大利麵 2把', '鹽 適量公克', '鮮磨黑胡椒 5公克', '羅勒葉 適量公克', '宮保辣椒段 5公克']
Index ID: 835
Name: 辣炒蒜味義大利麵
Description: 經典的蒜炒義大利麵結合中式乾辣椒段，激盪出獨特的辛香風味！乾辣椒段帶來微辣與濃郁椒香，搭配蒜香交織，讓橄欖油充分吸收香氣，完美滲透彈牙麵條，使每一口都辛香過癮、香辣夠味。簡單快炒，快速上桌，無論是日常餐點還是聚會料理，都能成為眾人矚目的美味焦點！
Ingredients: ['蒜頭 5顆', '橄欖油 8毫升', '義大利麵 2把', '鹽 適量公克', '鮮磨黑胡椒 5公克', '羅勒葉 適量公克', '宮保辣椒段 5公克']
Instructions: ['食材準備', '煮一鍋水將義大利麵煮至半熟，瀝乾水分撈出，以橄欖油拌勻備用。', '蒜頭切片，備用', '熱鍋，放入少許橄欖

In [91]:
print(llm_result)

推薦的料理是 **正港熱炒系列-鐵板黑胡椒**。

### 比較項目列表：
- 米酒
- 食用油
- 鮮磨黑胡椒
